## **Task 1**

In [1]:
# to access google drive folder
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

train_X = np.load("/content/drive/MyDrive/CSC3831/train_x.npy", allow_pickle=True)
train_y = np.load("/content/drive/MyDrive/CSC3831/train_y.npy", allow_pickle=True)

validate_X = np.load("/content/drive/MyDrive/CSC3831/valid_x.npy", allow_pickle=True)
validate_y = np.load("/content/drive/MyDrive/CSC3831/valid_y.npy", allow_pickle=True)

test_x = np.load("/content/drive/MyDrive/CSC3831/test_x.npy", allow_pickle=True)
test_y = np.load("/content/drive/MyDrive/CSC3831/test_y.npy", allow_pickle=True)


ModuleNotFoundError: No module named 'google.colab'

In [2]:
import matplotlib.pyplot as plt

def image_normalization(arr):
    return (arr - arr.min())/(arr.max()-arr.min())

def disable_ax_ticks(ax):
    ax.set_xticks([])
    ax.set_xticks([], minor=True)
    ax.set_yticks([])
    ax.set_yticks([], minor=True)

def show_mnist_examples(x, y):
    fig = plt.figure(constrained_layout=True,figsize=(12,9), dpi=100)
    gs = fig.add_gridspec(3,4)
    main_ax = fig.add_subplot(gs[:3,:3])
    fig.suptitle(y)
    #main_ax.imshow(np.moveaxis(x, 0, -1))# .imshow(x)
    main_ax.imshow(image_normalization(np.moveaxis(x, 0, -1)))
    disable_ax_ticks(main_ax)

    for j in range(3):
      c_ax = fig.add_subplot(gs[j,-1])
      subimage = x.copy()
      subimage[:j] = 0
      subimage[j+1:] = 0
      subimage[j] = subimage[j]-subimage[j].min()
      c_ax.imshow(image_normalization(np.moveaxis(subimage, 0, -1)))
      disable_ax_ticks(c_ax)
    plt.show()

# separating the color channels and reprocessing subimages
def separate_image(image):
  subimage_A = image[0]
  subimage_B = image[1]
  subimage_C = image[2]

  subimage_A = preprocess_image(subimage_A)
  subimage_B = preprocess_image(subimage_B)
  subimage_C = preprocess_image(subimage_C)

  return subimage_A, subimage_B, subimage_C

# scale to [0, 1] and reshape to (28, 28, 1)
def preprocess_model_data(image):
  image = image.astype("float32") / 255
  return np.expand_dims(image, -1)

# normalize and reshape to (28, 28, 1)
def preprocess_image(image):
  image = image_normalization(image)
  return np.expand_dims(image, -1)


# Pre-processing the data for training the model

In [7]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

batch_size = 128
epochs = 15

# load the data, split between train, validate and test sets
(x_model_train_org, y_model_train_org), (x_model_validate, y_model_validate) = mnist.load_data("mnist.npz")

number_test = round(x_model_train_org.shape[0] * 5/6)
x_model_train = x_model_train_org[:number_test]
y_model_train = y_model_train_org[:number_test]

x_model_test = x_model_train_org[number_test:]
y_model_test = y_model_train_org[number_test:]

x_model_train = preprocess_image(x_model_train)
x_model_validate = preprocess_image(x_model_validate)
x_model_test = preprocess_image(x_model_test)

# convert class vectors to one-hot-encoding
y_model_train = keras.utils.to_categorical(y_model_train, num_classes)
y_model_validate = keras.utils.to_categorical(y_model_validate, num_classes)
y_model_test = keras.utils.to_categorical(y_model_test, num_classes)

# Model 1

In [11]:
model = keras.Sequential() 
model.add(Conv2D(28, (3, 3), activation='relu', padding='same', input_shape=input_shape)) 
model.add(Conv2D(28, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

history = model.fit(x_model_train, y_model_train, batch_size=batch_size, epochs=epochs, validation_data=(x_model_validate,y_model_validate))
results = model.evaluate(x_model_test, y_model_test, batch_size=batch_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 28)        280       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 28)        7084      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 28)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 5488)              0         
                                                                 
 dense (Dense)               (None, 256)               1405184   
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                        

# Model 2

In [12]:
model = keras.Sequential() 
model.add(Conv2D(28, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1))) 
model.add(Conv2D(28, (3, 3), activation='relu', padding='same')) 
model.add(Flatten())  
model.add(Dense(128, activation='relu')) 
model.add(Dense(10, activation='softmax'))

model.summary()

batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(x_model_train, y_model_train, batch_size=batch_size, epochs=epochs, validation_data=(x_model_test,y_model_test))

results = model.evaluate(x_model_test, y_model_test, batch_size=batch_size)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 28)        280       
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 28)        7084      
                                                                 
 flatten_1 (Flatten)         (None, 21952)             0         
                                                                 
 dense_3 (Dense)             (None, 128)               2809984   
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
Total params: 2,818,638
Trainable params: 2,818,638
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
391/391 [========================

# Model 3

In [ ]:
model = keras.Sequential() 
model.add(Conv2D(28, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1))) 
model.add(Conv2D(28, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D((2, 2))) 
model.add(Flatten())
model.add(Dense(128, activation='relu')) 
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(x_model_train, y_model_train, batch_size=batch_size, epochs=epochs, validation_data=(x_model_test,y_
                                                                                                         
results = model.evaluate(x_model_test, y_model_test, batch_size=batch_size)model_test))

# Predict output for given images

In [ ]:
from random import randint 

iterations = 6
trace = []

for i  in range(iterations):
  correct_predictions = 0
  rounds = 1000

  for index in range(rounds):
    choice = randint(0,train_X.shape[0]-1)
    a, b, c = separate_image(train_X[choice])
    pred_A = model.predict(a.reshape(1, 28, 28, 1))
    pred_B = model.predict(b.reshape(1, 28, 28, 1))
    pred_C = model.predict(c.reshape(1, 28, 28, 1))
    if pred_A.argmax() + pred_B.argmax() + pred_C.argmax() - 1 == train_y[choice]:
      correct_predictions += 1

  print("{}/{} in {} seconds".format(i, iterations))
  print("correct predictions for", correct_predictions, "of", rounds, "tests")
